# Introduction

This notebook guides through calibration function development using published data (S. De Vito et al., Sensors and Actuators B: Chemical, Volume 129, Issue 2).

In [1]:
# Load libriaries.
import pandas as pd
import numpy as np
import tensorflow as tf
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
import tensorflow_probability as tfp
tfd = tfp.distributions
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error
import matplotlib.pyplot as plt
plt.rcParams['lines.linewidth'] = 1.0
plt.rcParams['font.size'] = 6.0
plt.rcParams['axes.titlesize'] = 6.0

# Additional Functions

Define some helper functions.

In [2]:
# Define helper functions.
scaler = StandardScaler() # Define standard scaler instance.
detector = IsolationForest(n_estimators=1000, behaviour="deprecated", contamination="auto", random_state=0) # Define outlier detector instance.
lin_reg = LinearRegression() # Define linear regression instance.
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x) # Define negative logarithmic likelihood.

# Preprocessing
Data is scaled before analysis. In particular, standard scaling provides better results with respect to R2 score and explained variance compared with unscaled analysis. No imputation is performed.

In [4]:
# Load data and keep only first six months.
data = pd.read_excel("data/AirQualityUCI.xlsx")
data = data[data["Date"] <= "2004-09-10"] # Keep only the first (drift-free) six months.

# Visualize data summary.
data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000
mean,-42.602261,1034.871853,-113.434193,2.874622,900.022746,59.912031,860.794701,34.725237,1557.502959,906.415216,16.023040,34.424144,-5.533866
std,83.726787,302.077005,193.392727,38.174135,312.298785,151.045512,298.766662,123.090508,411.689646,394.913480,40.798168,46.229254,36.066394
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.400000,918.250000,-200.000000,4.481885,736.750000,30.000000,723.062500,39.000000,1421.375000,670.062500,16.575000,28.275001,0.880123
50%,1.300000,1038.375000,-200.000000,8.224492,908.375000,85.000000,850.375000,81.000000,1576.750000,880.625000,23.000000,41.675000,1.080659
75%,2.200000,1176.937500,-200.000000,13.137205,1088.250000,149.000000,1014.375000,112.000000,1746.937500,1138.000000,28.350000,53.543750,1.355107
max,8.100000,2039.750000,1189.000000,40.260061,1776.250000,631.000000,1940.750000,233.000000,2746.000000,2474.750000,44.600000,85.150002,2.180639


In [5]:
# Select columns and remove rows with missing values.
columns = ["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH", "CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
data = data[columns].dropna(axis=0)

# Scale data to zero mean and unit variance.
X_t = scaler.fit_transform(data)

# Optional: Impute missing values.
# X_t = imputer.fit_transform(X_t)

# Remove outliers.
is_inlier = detector.fit_predict(X_t)
X_t = X_t[(is_inlier)>0,:]

# Restore frame.
dataset = pd.DataFrame(X_t, columns=columns)

# Select labels for inputs and outputs.
inputs = ["PT08.S1(CO)", "PT08.S3(NOx)", "PT08.S4(NO2)", "PT08.S5(O3)", "T", "AH"]
outputs = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]

# Data Handling
First, a data pipline is to be defined.

In [6]:
# Define some hyperparameters.
n_epochs = 50
n_samples = dataset.shape[0]
batch_size = np.floor(n_samples/10)
buffer_size = n_samples

# Define training and test data sizes.
n_train = int(0.7*dataset.shape[0])

# Define dataset instance.
data = tf.data.Dataset.from_tensor_slices((dataset[inputs].values, dataset[outputs].values)) # Define input/output data.
data = data.shuffle(n_samples, reshuffle_each_iteration=True) # Shuffle.

# Define train and test data instances.
data_train = data.take(n_train).batch(batch_size).repeat(n_epochs)
data_test = data.skip(n_train).batch(1).repeat(n_epochs)

# Training
A probabilistic neural network with two layers is used as machine learning algorithm. In a first iteration, only **aleotoric variability** is considered.

In [ ]:
# Define prior for Gaussians.
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(len(outputs), dtype=tf.float64), scale=1.0), reinterpreted_batch_ndims=1)

# Define model instance.
model = tfk.Sequential([
    tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"), # Input layer.
    tfk.layers.Dense(10, activation="relu", name="dense_1"), # First layer with ten nodes and non-linear activation.
    tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(len(outputs)), activation=None, name="distribution_weights"),
    tfp.layers.MultivariateNormalTriL(len(outputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_samples), name="output")
], name="model")

# Compile model.
model.compile(optimizer="adam", loss=neg_log_likelihood) # Negative logarithmic likelihood as loss function and ADAM optimizer.

# Run training session.
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)

# Describe model.
model.summary()

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


# Evaluation

In [ ]:
# Plot the training session.
fig = plt.figure(figsize=(2.95,2.95))
fig.tight_layout()
plt.xlim(0, np.round(len(model.history.epoch)+4,-1))
plt.ylim(-1, 5)
plt.xlabel("epoch")
plt.ylabel("negative logarithmic likelihood")
plt.plot(list(map(lambda x: x + 1, model.history.epoch)), model.history.history["loss"], label="training", color=plt.cm.viridis(0.3))
plt.plot(list(map(lambda x: x + 1, model.history.epoch)), model.history.history["val_loss"], label="validation", color=plt.cm.viridis(0.6))
plt.legend(frameon=False, loc=0)
plt.savefig('./figures/metrics_probabilistic_aleotoric',dpi=900,transparent=True,orientation='landscape',bbox_inches='tight')
plt.show()

In [ ]:
# Predict.
samples = 500
iterations = 10
test_iterator = tf.compat.v1.data.make_one_shot_iterator(data_test)
X_true, Y_true, Y_pred = np.empty(shape=(samples,len(inputs))), np.empty(shape=(samples,len(outputs))), np.empty(shape=(samples,len(outputs),iterations))
for i in range(samples):
    features, labels = test_iterator.get_next()
    X_true[i,:] = features
    Y_true[i,:] = labels.numpy()
    for k in range(iterations):
        Y_pred[i,:,k] = model.predict(features)
        
# Calculate mean and standard deviation.
Y_pred_m = np.mean(Y_pred, axis=-1)
Y_pred_s = np.std(Y_pred, axis=-1)

In [ ]:
# Determine slope.
slope = lin_reg.fit(Y_true.flatten().reshape(-1, 1), Y_pred_m.flatten().reshape(-1, 1))

# Perform predictions.
Y_syn = np.linspace(-10, 10, 10)
Y_syn_p = slope.predict(Y_syn.reshape(-1, 1))

# Plot predictions vs. true values.
fig = plt.figure(figsize=(2.95,2.95))
fig.tight_layout()
ax = fig.add_subplot(111)
ax.set_xlabel(r"Y$_{data}$")
ax.set_ylabel(r"Y$_{model}$")
ax.set_xlim(-4.0, 4.0)
ax.set_ylim(-4.0, 4.0)
ax.plot(Y_syn, Y_syn_p, ":", color="k", label=r"$\beta$ = "+str(np.round(slope.coef_[0][0],2)))
ax.plot([-10,10], [-10,10], "-", color="k")
ax.plot(Y_true.flatten(), Y_pred_m.flatten(), color=plt.cm.viridis(0.6), marker=".", alpha=1.0, mec='k', linestyle="")
plt.legend(frameon=False, loc=4, ncol=1)
plt.savefig('./figures/congruency_probabilistic_aleotoric',dpi=900,transparent=True,orientation='landscape',bbox_inches='tight')
plt.show()

# Training
A probabilistic neural network with two layers is used as machine learning algorithm. In a first iteration, **aleotoric and epistemic variability** is considered.

In [ ]:
# Define prior for Gaussians.
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(len(outputs), dtype=tf.float64), scale=1.0), reinterpreted_batch_ndims=1)

# Define model instance.
model = tfk.Sequential([
    tfk.layers.InputLayer(input_shape=(len(inputs),), name="input"), # Input layer.
    tfp.layers.DenseFlipout(10, activation="relu", name="dense_1"), # First layer with ten nodes and non-linear activation.
    tfp.layers.DenseFlipout(tfp.layers.MultivariateNormalTriL.params_size(len(outputs)), activation=None, name="distribution_weights"),
    tfp.layers.MultivariateNormalTriL(len(outputs), activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/n_samples), name="output")
], name="model")

# Compile model.
model.compile(optimizer="adam", loss=neg_log_likelihood) # MSE as loss function and ADAM optimizer.

# Run training session.
model.fit(data_train, epochs=n_epochs, validation_data=data_test, verbose=False)

# Describe model.
model.summary()

In [ ]:
# Plot the training session.
fig = plt.figure(figsize=(2.95,2.95))
fig.tight_layout()
plt.xlim(0, np.round(len(model.history.epoch)+4,-1))
plt.ylim(0, 500)
plt.xlabel("epoch")
plt.ylabel("negative logarithmic likelihood")
plt.plot(list(map(lambda x: x + 1, model.history.epoch)), model.history.history["loss"], label="training loss", color=plt.cm.viridis(0.3))
plt.plot(list(map(lambda x: x + 1, model.history.epoch)), model.history.history["val_loss"], label="validation loss", color=plt.cm.viridis(0.6))
plt.legend(frameon=False, loc=0)
plt.savefig('./figures/metrics_probabilistic_aleotoric_epistemic',dpi=900,transparent=True,orientation='landscape',bbox_inches='tight')
plt.show()